In [1]:
import os
import torch
import json
import argparse
from tqdm import tqdm
import torch.optim as optim
from torchvision.utils import save_image
from PIL import Image

# from training_models import MyClipEnsemble, MyInternEnsemble

from datasets import collate_fn_image, ImageDataset, EnsembleImageDataset, AdvDiffVLMImageDataset
# from poison_utils import L2_norm

from torchvision import transforms

# Image Training

In [2]:
dataset_name = "mini_MathVista_grid"
method = "i2i_AdvDiffVLMAttack"
args = {
    "model": "clip",
    "poison_save_pth": f"./data/poisons/{dataset_name}+{method}",
    "iter_attack": 200,
    "lr_attack": 1,
    # "base_data_pth": f"data/{dataset_name}/base",
    "base_data_pth": f"data/{dataset_name}/base_512",
    "target_data_pth": f"data/{dataset_name}/target",
    "questions_pth": f"data/{dataset_name}/questions.json",
    "temperature": 0,
    "max_new_tokens": 200,
    "eps": 16,
}


def read_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


In [3]:
# encoder_model = MyClip()
# encoder_model = MyClipEnsemble()
# encoder_model = MyInternEnsemble()
# device = encoder_model.device
device = "cuda"

question_json = read_json(args["questions_pth"])
pids = question_json.keys()



base_caps = read_json(os.path.join(args["base_data_pth"], "caps.json"))
target_caps = read_json(os.path.join(args["target_data_pth"], "caps.json"))

In [4]:
# base_dataset = ImageDataset(base_caps["annotations"])
# target_dataset = ImageDataset(target_caps["annotations"])
# base_dataset = EnsembleImageDataset(base_caps["annotations"])
# target_dataset = EnsembleImageDataset(target_caps["annotations"])
base_dataset = AdvDiffVLMImageDataset(base_caps["annotations"])
target_dataset = AdvDiffVLMImageDataset(target_caps["annotations"])

base_dataloader = torch.utils.data.DataLoader(
    base_dataset, batch_size=1, 
    shuffle=False, collate_fn=collate_fn_image
)

target_dataloader = torch.utils.data.DataLoader(
    target_dataset, batch_size=1, 
    shuffle=False, collate_fn=collate_fn_image
)

In [5]:
import matplotlib.pyplot as plt
def show_img(img, caption):
    print(caption)
    plt.imshow(img.squeeze(0).permute(1,2,0))
    plt.axis("off")
    plt.show()

In [6]:
def i2i_attack(model, image_base, image_target, iters=100, lr=1/255, eps=8):
      '''
      optimizing x_adv to minimize emb_dist( img_embed of x_adv, img_embed of image_target ) within Lp constraint using PGD

      model: model class with image embedding functionality (e.g. CLIP, EVA)
      image_base, image_target: images between [0,1] 
      emb_dist: the distance metrics for vision embedding (such as L2): take a batch of bs image pairs as input, \
            and output EACH of pair-wise distances of the whole batch (size = [bs])

      eps: for Lp constraint
      lr: the step size. The update is grad.sign * lr
      diff_aug: using differentiable augmentation, e.g. RandomResizeCrop

      return: X_adv between [0,1]
      '''

      bs = image_base.size(0)
      device = image_base.device

      with torch.no_grad():
            embedding_targets = model.encode_image(image_target)
            embedding_targets = embedding_targets / embedding_targets.norm(dim=-1, keepdim=True)

      X_adv = image_base.clone().detach()
      X_adv.requires_grad_(True) 

      loss_best = 1e8
      X_adv_best = X_adv.clone().detach()

      # for i in tqdm(range(iters), leave=False):
      for i in range(iters):
            embs = model.encode_image(X_adv)
            embs = embs / embs.norm(dim=-1, keepdim=True)  # Normalize embeddings

            loss = L2_norm(embs, embedding_targets)
            # loss = torch.mean(torch.sum(embs * embedding_targets, dim=1))

            if loss < loss_best:
                  loss_best = loss.clone().detach()
                  X_adv_best = X_adv.clone().detach()

            # optimizer.zero_grad()
            loss.backward()

            # print losses at increments
            if i% max(int(iters/20),1) == 0:
                  # print('Iter :{} loss:{:.4f}, lr * 255:{:.4f}'.format(i,loss.item()/bs, scheduler.get_last_lr()[0]*255))
                  print('Iter :{} loss:{:.4f}, lr * 255:{:.4f}'.format(i,loss.item()/bs, lr*255))

            grad = X_adv.grad
            assert grad != None
            grad = grad / torch.mean(torch.abs(grad), dim=(1,2,3), keepdim=True)           
            
            perturbation = lr * grad.sign()
            # print(perturbation.shape)
            X_adv.data = X_adv.data.detach() + perturbation
            X_adv.data = torch.minimum(torch.maximum(X_adv, image_base - eps), image_base + eps) 
            X_adv.data = X_adv.data.clamp(0,255)
            X_adv.grad = None  

            if torch.isnan(loss):
                  print('Encounter nan loss at iteration {}'.format(i))
                  break                 

      print('Best Total loss:{:.4f}'.format(loss.item()))

      return X_adv_best, loss_best




In [7]:
def SGAImageAttack(model, image, cap, iters=100, lr=1/255, eps=8):
      '''
      optimizing x_adv to maximize emb_dist( img_embed of x_adv, text_embed of captions ) within Lp constraint using PGD

      model: model class with image and text embedding functionality (e.g. CLIP, EVA)
      image, cap: image between [0,255] (float) caption dict for that image
      emb_dist: the distance metrics for vision embedding (such as L2): take a batch of bs image pairs as input, \
            and output EACH of pair-wise distances of the whole batch (size = [bs])

      eps: for Lp constraint
      lr: the step size. The update is grad.sign * lr

      return: X_adv between [0,255]
      '''

      bs = image.size(0)
      device = image.device

      with torch.no_grad():
            embedding_targets = model.encode_text(cap['caption'])
            embedding_targets = embedding_targets / embedding_targets.norm(dim=-1, keepdim=True)

      X_adv = image.clone().detach()
      X_adv.requires_grad_(True) 

      loss_best = 1e8
      X_adv_best = X_adv.clone().detach()

      # for i in tqdm(range(iters), leave=False):
      for i in range(iters):
            embs = model.encode_image(X_adv)
            embs = embs / embs.norm(dim=-1, keepdim=True)  # Normalize embeddings

            loss = -L2_norm(embs, embedding_targets)

            if loss < loss_best:
                  loss_best = loss.clone().detach()
                  X_adv_best = X_adv.clone().detach()

            loss.backward()

            # print losses at increments
            if i% max(int(iters/20),1) == 0:
                  print('Iter :{} loss:{:.4f}, lr * 255:{:.4f}'.format(i,loss.item()/bs, lr*255))

            grad = X_adv.grad
            assert grad != None
            grad = grad / torch.mean(torch.abs(grad), dim=(1,2,3), keepdim=True)           
            
            perturbation = lr * grad.sign()
            # print(perturbation.shape)
            X_adv.data = X_adv.data.detach() - perturbation
            X_adv.data = torch.minimum(torch.maximum(X_adv, image_base - eps), image_base + eps) 
            X_adv.data = X_adv.data.clamp(0,255)
            X_adv.grad = None  

            if torch.isnan(loss):
                  print('Encounter nan loss at iteration {}'.format(i))
                  break                 

      print('Best Total loss:{:.4f}'.format(loss.item()))

      return X_adv_best, loss_best

In [8]:
def i2i_EnsembleAttack(model, image, target, iters=100, lr=1/255, eps=8):
      '''
      optimizing x_adv to maximize emb_dist( img_embed of x_adv, text_embed of captions ) within Lp constraint using PGD

      model: model class with image and text embedding functionality (e.g. CLIP, EVA)
      image, cap: image between [0,255] (float) caption dict for that image
      emb_dist: the distance metrics for vision embedding (such as L2): take a batch of bs image pairs as input, \
            and output EACH of pair-wise distances of the whole batch (size = [bs])

      eps: for Lp constraint
      lr: the step size. The update is grad.sign * lr

      return: X_adv between [0,255]
      '''

      bs = image.size(0)
      device = image.device

      with torch.no_grad():
            embedding_targets = model.encode_image(target)

      X_adv = image.clone().detach()
      X_adv.requires_grad_(True) 

      loss_best = -1e8
      X_adv_best = X_adv.clone().detach()

      # for i in tqdm(range(iters), leave=False):
      for i in range(iters):
            embs = model.encode_image(X_adv, use_grad=True)

            # loss = L2_norm(embs, embedding_targets)
            grad, loss = model.get_gradients(embs, embedding_targets, X_adv) 

            torch.cuda.empty_cache()
            if loss > loss_best:
                  loss_best = loss.clone().detach()
                  X_adv_best = X_adv.clone().detach()

            # print losses at increments
            if i% max(int(iters/20),1) == 0:
                  print('Iter :{} loss:{:.4f}, lr:{:.4f}'.format(i,loss.item()/bs, lr))

            
            perturbation = lr * grad.sign()
            # print(perturbation.shape)
            with torch.no_grad():
                  X_adv = X_adv + perturbation
                  X_adv = torch.min(torch.max(X_adv, image_base - eps), image_base + eps)
                  X_adv = torch.clamp(X_adv, 0, 255)
            X_adv.requires_grad_(True)

            if torch.isnan(loss):
                  print('Encounter nan loss at iteration {}'.format(i))
                  break                 

      print('Best Total loss:{:.4f}'.format(loss.item()))

      # return X_adv.clone().detach(), loss_best
      return X_adv_best, loss_best

In [9]:
# loss gets logits and maximizes entropy of each generated word for image caption

# def MaximizeInformationEntropyAttack(model, image, cap, iters=100, lr=1/255, eps=8):


### SGA Image Attack

In [10]:
# import matplotlib.pyplot as plt

# ###### Running SGA Image Attack ######
# original_sizes_list = []
# X_adv_list = []
# all_losses = []
# loss_attack_list = []

# target_path = os.path.join(args['poison_save_pth'], "SGA")

# if not os.path.exists(target_path):
#     os.makedirs(target_path)
# saved_images = set(os.listdir(target_path))
# saved_image_ids = {int(fname.split('.')[0]) for fname in saved_images if fname.endswith(('.png', '.jpg', '.jpeg'))}


# for i, (image_base, base_cap) in  tqdm(enumerate(base_dataloader), desc=f"Processing all images", total=len(base_dataloader)):
#     if base_cap['name'] in saved_image_ids:
#             print(f"{base_cap['name']} already processed for {target_cap['name']}, skipping...")
#             continue

#     print('image name = ', base_cap['name'])
#     image_base = image_base.to(device)

#     X_adv, loss_attack = SGAImageAttack(
#             model=encoder_model,
#             image=image_base,
#             cap=base_cap,
#             iters=args['iter_attack'],
#             lr=args['lr_attack'],
#             eps=args['eps']
#     )


#     ###### Save poisoned images after each batch ######
#     img_pth = os.path.join(target_path, f"{base_cap['name']}.png")
#     image_to_save = X_adv/255.0
#     print()
#     if base_cap['name'] not in saved_image_ids:  # Only save if it doesn't already exist
#         print("Max Pixel Difference between Adversarial Image and Base *255:", round(torch.max(torch.abs(X_adv-image_base)).item(), 4))
#         save_image(image_to_save.cpu(), img_pth)
#         print(f"Saved poisoned image {base_cap['name']} to {img_pth}") # mathvista

#     # show_img(((X_adv-image_base).to(torch.uint8).cpu()*10), ("Adversarial Noise", img_pth))
#     # show_img(image_to_save.cpu(), ("Adversarial Image", img_pth))

        

# print("Done.")

### i2i attack

In [11]:
# ##### Running i2i attack ######
# original_sizes_list = []
# X_adv_list = []
# all_losses = []
# loss_attack_list = []
# start_idx = 0
# end_idx = 0
# # for j, (image_target, target_cap) in  tqdm(enumerate(target_dataloader)):
# for j, (image_target, target_cap) in  tqdm(enumerate(target_dataloader), desc="Target Dataloader", position=0, leave=True):
#     target_name = target_cap["name"]
#     target_path = os.path.join(args["poison_save_pth"], target_cap["name"])
#     if not os.path.exists(target_path):
#         os.makedirs(target_path)

#     ###### Resume by checking already saved images ######
#     saved_images = set(os.listdir(target_path))
#     saved_image_ids = {int(fname.split('.')[0]) for fname in saved_images if fname.endswith(('.png', '.jpg', '.jpeg'))}

#     # show_img(image_target, ("Working on target:", target_name))

#     for i, (image_base, base_cap) in  tqdm(enumerate(base_dataloader), desc=f"Processing {target_name}", total=len(base_dataloader), position=1, leave=False):
#         if base_cap['name'] in saved_image_ids:
#                 print(f"{base_cap['name']} already processed for {target_cap['name']}, skipping...")
#                 continue

#         print('image name = ', base_cap['name'])
#         image_base, image_target = image_base.to(device), image_target.to(device)

#         X_adv, loss_attack = i2i_attack(
#                 model=encoder_model,
#                 image_base=image_base,
#                 image_target=image_target,
#                 iters=args['iter_attack'],
#                 lr=args['lr_attack'],
#                 eps=args['eps']
#         )


#         ###### Save poisoned images after each batch ######
#         img_pth = os.path.join(target_path, f"{base_cap['name']}.png")
#         image_to_save = X_adv/255.0
#         print()
#         if base_cap['name'] not in saved_image_ids:  # Only save if it doesn't already exist
#             print("Max Pixel Difference between Adversarial Image and Base *255:", round(torch.max(torch.abs(X_adv-image_base)).item(), 4))
#             save_image(image_to_save.cpu(), img_pth)
#             print(f"Saved poisoned image {base_cap['name']} to {img_pth}") # mathvista

#         # show_img(((X_adv-image_base).to(torch.uint8).cpu()*10), ("Adversarial Noise", img_pth))
#         # show_img(image_to_save.cpu(), ("Adversarial Image", img_pth))

        

# print("Done.")

## i2i Ensemble attack

In [12]:
# ##### Running i2i attack ######
# original_sizes_list = []
# X_adv_list = []
# all_losses = []
# loss_attack_list = []
# start_idx = 0
# end_idx = 0
# # for j, (image_target, target_cap) in  tqdm(enumerate(target_dataloader)):
# for j, (image_target, target_cap) in  tqdm(enumerate(target_dataloader), desc="Target Dataloader", position=0, leave=True):
#     target_name = target_cap["name"]
#     target_path = os.path.join(args["poison_save_pth"], target_cap["name"])
#     if not os.path.exists(target_path):
#         os.makedirs(target_path)

#     ###### Resume by checking already saved images ######
#     saved_images = set(os.listdir(target_path))
#     saved_image_ids = {fname.split('.')[0] for fname in saved_images if fname.endswith(('.png', '.jpg', '.jpeg'))}
#     if all(name in saved_image_ids for name in [base_cap['name'] for (_, base_cap) in base_dataloader]):
#         print(f"{target_cap['name']} entirely complete")
#         continue

#     # show_img(image_target, ("Working on target:", target_name))

#     for i, (image_base, base_cap) in  tqdm(enumerate(base_dataloader), desc=f"Processing {target_name}", total=len(base_dataloader), position=1, leave=False):
#         if base_cap['name'] in saved_image_ids:
#                 print(f"{base_cap['name']} already processed for {target_cap['name']}, skipping...")
#                 continue

#         print('image name = ', base_cap['name'])
#         image_base, image_target = image_base.to(device), image_target.to(device)

#         X_adv, loss_attack = i2i_EnsembleAttack(
#                 model=encoder_model,
#                 image=image_base,
#                 target=image_target,
#                 iters=args['iter_attack'],
#                 lr=args['lr_attack'],
#                 eps=args['eps']
#         )


#         ###### Save poisoned images after each batch ######
#         img_pth = os.path.join(target_path, f"{base_cap['name']}.png")
#         image_to_save = X_adv / 255.0
#         print()
#         if base_cap['name'] not in saved_image_ids:  # Only save if it doesn't already exist
#             print("Max Pixel Difference between Adversarial Image and Base *255:", torch.max(torch.abs(X_adv-image_base)).item())
#             save_image(image_to_save.cpu(), img_pth)
#             print(f"Saved poisoned image {base_cap['name']} to {img_pth}") # mathvista

#         # show_img(((X_adv-image_base).to(torch.uint8).cpu()*10), ("Adversarial Noise", img_pth))
#         # show_img(image_to_save.cpu(), ("Adversarial Image", img_pth))

        

# print("Done.")

In [13]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import cv2
from torchvision import models, transforms
from PIL import Image

# ----------------- GradCAM++ Implementation -----------------
class GradCAMPlusPlus:
    def __init__(self, model, target_layer):
        self.model = model
        self.model.eval()
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None

        # Register hooks to save gradients and activations
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output

    def save_gradient(self, module, grad_in, grad_out):
        self.gradients = grad_out[0]

    def forward(self, x):
        return self.model(x)

    def generate_cam(self, target_class):
        # Compute weights based on Grad-CAM++ formulation
        gradients = self.gradients[0].cpu().data.numpy()
        activations = self.activations[0].cpu().data.numpy()
        
        alpha_num = gradients ** 2
        alpha_denom = gradients ** 2 * 2 + np.sum(activations * (gradients ** 3), axis=(1, 2), keepdims=True)
        # Avoid division by zero
        alpha_denom = np.where(alpha_denom != 0, alpha_denom, np.ones(alpha_denom.shape))
        alphas = alpha_num / alpha_denom
        weights = np.maximum(gradients, 0) * alphas
        weights = np.sum(weights, axis=(1, 2))
        cam = np.sum(weights[:, np.newaxis, np.newaxis] * activations, axis=0)
        cam = np.maximum(cam, 0)  # ReLU
        # Resize CAM to 224x224 (same as input image size)
        cam = cv2.resize(cam, (224, 224))
        # Normalize CAM to [0, 1]
        cam = cam - np.min(cam)
        if np.max(cam) != 0:
            cam = cam / np.max(cam)
        return cam

    def __call__(self, input_tensor, target_class):
        # Forward pass
        output = self.forward(input_tensor)
        # Zero gradients and backward pass with respect to target class
        self.model.zero_grad()
        class_loss = output[0, target_class]
        class_loss.backward()
        # Generate CAM
        cam = self.generate_cam(target_class)
        return cam

# ----------------- Preprocessing function -----------------
def preprocess_image(img_path):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(img_path).convert("RGB")
    img_tensor = preprocess(img).unsqueeze(0)
    return img, img_tensor

# ----------------- Main Script -----------------

# Define paths
cle_data_dir = os.path.join(args['base_data_pth'])       # directory containing subdirectories {num} ahhhhh
mask_output_dir = os.path.join(args['base_data_pth']+"_masked")    # directory where binary masks will be saved

# Create output directory if it doesn't exist
if not os.path.exists(mask_output_dir):
    os.makedirs(mask_output_dir)

# Load pretrained model (ResNet50) and specify the target convolutional layer
model = models.resnet50(pretrained=True)
target_layer = model.layer4[-1]  # last conv layer of ResNet50
grad_cam_pp = GradCAMPlusPlus(model, target_layer)

# Process each image file in this subdirectory (assume common image extensions)
for filename in os.listdir(cle_data_dir):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        continue  # skip non-image files
    
    img_path = os.path.join(cle_data_dir, filename)
    try:
        original_img, input_tensor = preprocess_image(img_path)
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        continue
    
    # Forward pass through the model to get prediction
    with torch.no_grad():
        output = model(input_tensor)
    target_class = output.argmax().item()  # or you can specify a target class if needed
    
    # Generate GradCAM++ heatmap for the target class
    cam = grad_cam_pp(input_tensor, target_class)
    
    # Convert heatmap to binary mask using thresholding (e.g., threshold at 0.5)
    binary_mask = (cam > 0.5).astype(np.uint8) * 255  # 255 for white regions
    
    # Optionally, convert to PIL Image (if you want to save in common formats)
    mask_img = Image.fromarray(binary_mask)
    
    # Save binary mask image with a modified filename
    base_name, _ = os.path.splitext(filename)
    mask_filename = f"{base_name}_mask.png"
    mask_save_path = os.path.join(mask_output_dir, mask_filename)
    mask_img.save(mask_save_path)
    print(f"Saved binary mask for {img_path} to {mask_save_path}")


/home/users/mxy/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/users/mxy/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


/home/users/mxy/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Saved binary mask for data/mini_MathVista_grid/base_512/1.jpg to data/mini_MathVista_grid/base_512_masked/1_mask.png
Saved binary mask for data/mini_MathVista_grid/base_512/10.jpg to data/mini_MathVista_grid/base_512_masked/10_mask.png
Saved binary mask for data/mini_MathVista_grid/base_512/100.jpg to data/mini_MathVista_grid/base_512_masked/100_mask.png
Saved binary mask for data/mini_MathVista_grid/base_512/11.jpg to data/mini_MathVista_grid/base_512_masked/11_mask.png
Saved binary mask for data/mini_MathVista_grid/base_512/12.jpg to data/mini_MathVista_grid/base_512_masked/12_mask.png
Saved binary mask for data/mini_MathVista_grid/base_512/13.jpg to data/mini_MathVista_grid/base_512_masked/13_mask.png
Saved binary mask for data/mini_MathVista_grid/base_512/14.jpg to data/mini_MathVista_grid/base_512_masked/14_mask.png
Saved binary mask for data/mini_MathVista_grid/base_512/15.jpg to data/mini_MathVista_grid/base_512_masked/15_mask.png
Saved binary mask for data/mini_MathVista_grid/b

In [14]:
import sys
import os 
import random

os.environ["CUDA_VISIBLE_DEVICES"]="0"

sys.path.append(".")
sys.path.append('./taming-transformers')
from taming.models import vqgan 

import torch
from omegaconf import OmegaConf

from ldm.util import instantiate_from_config

from ldm.models.diffusion.ddim_main import DDIMSampler
from torchvision.models import resnet50, ResNet50_Weights

from torchvision.utils import save_image
from torch.backends import cudnn
import numpy as np
import time



def to_tensor(pic):
    mode_to_nptype = {"I": np.int32, "I;16": np.int16, "F": np.float32}
    img = torch.from_numpy(np.array(pic, mode_to_nptype.get(pic.mode, np.uint8), copy=True))
    img = img.view(pic.size[1], pic.size[0], len(pic.getbands()))
    img = img.permute((2, 0, 1)).contiguous()
    return img.to(dtype=torch.get_default_dtype())

def load_model_from_config(config, ckpt):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt)#, map_location="cpu")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    model.cuda()
    model.eval()
    return model

# def get_model(): 
#     config = OmegaConf.load("ldm.yaml")  # configs/latent-diffusion/cin256-v2.yaml
#     model = load_model_from_config(config, "model.ckpt")  # https://ommer-lab.com/files/latent-diffusion/nitro/cin/model.ckpt
def get_model():  # 下载地址
    config = OmegaConf.load("configs/latent-diffusion/cin256-v2.yaml")  
    model = load_model_from_config(config, "models/ldm/cin256-v2/model.ckpt")
    return model

## 设置随机种子

seed = 0
cudnn.benchmark = False
cudnn.deterministic = True
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
# seed for everything
# credit: https://www.kaggle.com/code/rhythmcam/random-seed-everything
DEFAULT_RANDOM_SEED = 0
device = "cuda" if torch.cuda.is_available() else "cpu"

# basic random seed
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

# torch random seed
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# combine
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

seedEverything()
#------------------------------------------------------------------ #  

model = get_model()  # 扩散模型

### CLIP模型
import clip 
import torchvision
device = model.device

clip_rn_50,_ = clip.load('RN50', device=device)
clip_rn_101,_ = clip.load('RN101', device=device)
clip_vit_b_16,_ = clip.load('ViT-B/16', device=device)
clip_vit_b_32,_ = clip.load('ViT-B/32', device=device)
clip_vit_l_14,_ = clip.load('ViT-L/14', device=device)
models = [clip_rn_50, clip_rn_101, clip_vit_b_16, clip_vit_b_32]
# models = [clip_vit_b_32]
clip_preprocess = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(clip_vit_b_32.visual.input_resolution, interpolation=torchvision.transforms.InterpolationMode.BICUBIC, antialias=True),
        # torchvision.transforms.Lambda(lambda img: torch.clamp(img, 0.0, 255.0) / 255.0),
        torchvision.transforms.CenterCrop(clip_vit_b_32.visual.input_resolution),
        torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)), # CLIP imgs mean and std.
    ]
)
final_preprocess = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(clip_vit_b_32.visual.input_resolution, interpolation=torchvision.transforms.InterpolationMode.BICUBIC, antialias=True),
        # torchvision.transforms.Lambda(lambda img: torch.clamp(img, 0.0, 255.0) / 255.0),
        torchvision.transforms.CenterCrop(clip_vit_b_32.visual.input_resolution),
    ]
)


class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index: int):
        original_tuple = super().__getitem__(index)
        path, _ = self.samples[index]
        return original_tuple + (path,)

 # 读取数据
input_res = 224
# 建立映射关系


batch_size = 1
num_samples = 100
transform_fn = torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize(224, interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
            torchvision.transforms.CenterCrop(224),
            torchvision.transforms.Lambda(lambda img: img.convert("RGB")),
            torchvision.transforms.Lambda(lambda img: to_tensor(img)),
            torchvision.transforms.Lambda(lambda img: torch.clamp(img, 0.0, 255.0) / 255.0),
        ]
    )
transform_fn_org = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(256, interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
        torchvision.transforms.CenterCrop(256),
        torchvision.transforms.ToTensor(), # [0, 1]
        torchvision.transforms.Lambda(lambda img: (img * 2 - 1)),
        # torchvision.transforms.Lambda(lambda img: img.convert("RGB")),
        # torchvision.transforms.Lambda(lambda img: to_tensor(img)),
    ]
)

sampler = DDIMSampler(model, models=models, preprocess=clip_preprocess)  # 使用ddim采样器


import numpy as np 
from PIL import Image
from einops import rearrange  # 用于重新排列Tensor的维度
import cv2

classes = [2]  # 定义要生成的类
n_samples_per_class = 1
# 读取类别
import pandas as pd

ddim_steps = 200 # 200
ddim_eta = 0.0
scale = 5.0   # for unconditional guidance

img_transformed_list = []
cam_root = args['base_data_pth']+"_masked"

for j, (image_target, target_cap) in  tqdm(enumerate(target_dataloader), desc="Target Dataloader", position=0, leave=True):
    target_name = target_cap["name"]
    target_path = os.path.join(args["poison_save_pth"], target_cap["name"])
    if not os.path.exists(target_path):
        os.makedirs(target_path)

    ###### Resume by checking already saved images ######
    saved_images = set(os.listdir(target_path))
    saved_image_ids = {fname.split('.')[0] for fname in saved_images if fname.endswith(('.png', '.jpg', '.jpeg'))}
    if all(name in saved_image_ids for name in [base_cap['name'] for (_, base_cap) in base_dataloader]):
        print(f"{target_cap['name']} entirely complete")
        continue
    for i, (image_base, base_cap) in  tqdm(enumerate(base_dataloader), desc=f"Processing {target_name}", total=len(base_dataloader), position=1, leave=False):
        if base_cap['name'] in saved_image_ids:
                print(f"{base_cap['name']} already processed for {target_cap['name']}, skipping...")
                continue

        print('image name = ', base_cap['name'])
        image_org, image_tgt = image_base.to(device), image_target.to(device)

        start = time.time()

        # # get tgt featutres
        with torch.no_grad():
            tgt_image_features_list=[]
            image_tgt = clip_preprocess(image_tgt)
            for clip_model in models:
                tgt_image_features = clip_model.encode_image(image_tgt)  # [bs, 512]
                tgt_image_features = tgt_image_features / tgt_image_features.norm(dim=1, keepdim=True)
                tgt_image_features_list.append(tgt_image_features)
        
        with torch.no_grad():
            with model.ema_scope():  # 指数平滑
                uc = model.get_learned_conditioning(
                    {model.cond_stage_key: torch.tensor(n_samples_per_class*[1000]).to(model.device)}
                    ) # 类别为0-999，因此无条件用1000表示
                # label_id = path[0].split('/')[-2]
                # label_id = name_key[label_id] # 用于映射
                # class_label = labeles[label_id]
                # print("class_label:", class_label)
                # for class_label in classes:
                all_samples = list()
                all_labels = list() 
                # print(f"rendering {n_samples_per_class} examples of class '{class_label}' in {ddim_steps} steps and using s={scale:.2f}.")
                xc = torch.tensor(n_samples_per_class*[0]) #### IDK WHAT THIS DOES BE WARNED AND PROBABLY EDIT LATER AHHHHHH
                c = model.get_learned_conditioning({model.cond_stage_key: xc.to(model.device)})
                encoder_posterior = model.encode_first_stage(image_org)
                z = model.get_first_stage_encoding(encoder_posterior).detach()
                cam = cv2.imread(os.path.join(cam_root, base_cap['name']+'_mask.png'), 0) / 255.
                cam = cv2.resize(cam, (128, 128)) # CHANGED TO 128,128 FROM 64,64. WORKS NOT SURE WHY. NOT SURE HOW IT TRANSFORMS IT
                cam = torch.tensor(cam).float()

                # print("size of mask:", cam.size())
                # print("size of tgt image feats:", tgt_image_features_list[0].size())
                
                samples_ddim, _ = sampler.sample(S=ddim_steps,
                                                conditioning=c,
                                                # conditioning=None,
                                                x_T=z,
                                                batch_size=n_samples_per_class,
                                                shape=[3, 64, 64],
                                                verbose=False,
                                                unconditional_guidance_scale=scale,
                                                unconditional_conditioning=uc, 
                                                eta=ddim_eta,
                                                label=xc.to(model.device),
                                                # label=None,
                                                tgt_image_features_list=tgt_image_features_list,
                                                org_image_features_list=None,
                                                cam=cam,
                                                K=1,s=35,a=5)
                
                for _ in range(1):
                    samples_ddim, _ = sampler.sample(S=ddim_steps,
                                                conditioning=c,
                                                x_T=samples_ddim,
                                                batch_size=n_samples_per_class,
                                                shape=[3, 64, 64],
                                                verbose=False,
                                                unconditional_guidance_scale=scale,
                                                unconditional_conditioning=uc, 
                                                eta=ddim_eta,
                                                label=xc.to(model.device),
                                                tgt_image_features_list=tgt_image_features_list,
                                                org_image_features_list=None,
                                                cam=cam,
                                                K=1,s=35,a=5)
                
                x_samples_ddim = model.decode_first_stage(samples_ddim)
                x_samples_ddim = torch.clamp((x_samples_ddim+1.0)/2.0, 
                                            min=0.0, max=1.0)


        img_transformed = clip_preprocess(x_samples_ddim).to(device) # image transformation to model input
        img_transformed_list.append(x_samples_ddim)
        adv_image_feature_list = []
        for clip_model in models:
            adv_image_features = clip_model.encode_image(img_transformed)
            adv_image_features = adv_image_features / adv_image_features.norm(dim=1, keepdim=True)
            adv_image_feature_list.append(adv_image_features)
        crit_list = []
        for pred_i, target_i in zip(adv_image_feature_list, tgt_image_features_list):
            crit =  torch.mean(torch.sum(pred_i * target_i, dim=1))  # 有目标攻击
            crit_list.append(crit.data.detach().cpu().numpy())
        print("loss:", crit_list)

        ###### Save poisoned images after each batch ######
        img_pth = os.path.join(target_path, f"{base_cap['name']}.png")
        # image_to_save = X_adv / 255.0
        # print()
        if base_cap['name'] not in saved_image_ids:  # Only save if it doesn't already exist
            # print("Max Pixel Difference between Adversarial Image and Base *255:", torch.max(torch.abs(X_adv-image_base)).item())
            save_image(x_samples_ddim, img_pth)
            print(f"Saved poisoned image {base_cap['name']} to {img_pth}") # mathvista

        end = time.time()
        print("Time:", end-start)
        
        # for path_idx in range(len(path)):
        #     folder, name = path[path_idx].split("/")[-2], path[path_idx].split("/")[-1]
        #     # folder_to_save = os.path.join('../_output_img', , folder)
        #     folder_to_save = "/data/gq/result/adversarial_attack/dda/result/second_paper/samples"
        #     if not os.path.exists(folder_to_save):
        #         os.makedirs(folder_to_save, exist_ok=True)
        #     if 'JPEG' in name:
        #         torchvision.utils.save_image(final_preprocess(x_samples_ddim)[path_idx], os.path.join(folder_to_save, name[:-4]) + 'png')
        #     elif 'png' in name:
        #         torchvision.utils.save_image(final_preprocess(x_samples_ddim)[path_idx], os.path.join(folder_to_save, name))

Loading model from models/ldm/cin256-v2/model.ckpt


/home/users/mxy/.local/lib/python3.10/site-packages/pytorch_lightning/core/lightning.py:2058: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook


LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 400.92 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels


Target Dataloader: 1it [00:00,  2.81it/s]

bar entirely complete


...

Data shape for DDIM sampling is (1, 3, 64, 64), eta 0.0
Running Adversarial Sampling at 0 step
Running DDIM Sampling with 200 timesteps







DDIM Sampler: 100%|██████████| 200/200 [00:23<00:00,  8.53it/s]


loss: [array(0.6323, dtype=float16), array(0.7827, dtype=float16), array(0.7295, dtype=float16), array(0.6465, dtype=float16)]
Saved poisoned image 54 to ./data/poisons/mini_MathVista_grid+i2i_AdvDiffVLMAttack/pie/54.png
Time: 47.28971028327942
image name =  28
Data shape for DDIM sampling is (1, 3, 64, 64), eta 0.0
Running Adversarial Sampling at 0 step
Running DDIM Sampling with 200 timesteps









DDIM Sampler: 100%|██████████| 200/200 [00:23<00:00,  8.47it/s]


Data shape for DDIM sampling is (1, 3, 64, 64), eta 0.0
Running Adversarial Sampling at 0 step
Running DDIM Sampling with 200 timesteps


In [ ]:
raise Exception("Here to stop execution for this section")

# Mathvista Evaluation Pipeline

### Generate Response

In [ ]:
attacking_model="internvl"
captioning_model = "gpt-3.5-turbo"
method = "i2i_EnsembleAttack"

In [ ]:
import os
import io
import time
import argparse

from tqdm import tqdm

import sys
from poison_utils import *

from eval_models import internlm, internvl
from eval_models import gpt

from build_query import create_query_data



In [ ]:


def verify_response(response):
    if isinstance(response, str):
        response = response.strip() 
    if response == "" or response == None:
        return False
    if "Response Error" in response:
        return False
    return True


def evaluate_code(code_string):
    # execute_code_and_capture_output
    # Backup the original stdout
    old_stdout = sys.stdout
    
    # Redirect stdout to capture the output
    new_stdout = io.StringIO()
    sys.stdout = new_stdout
    
    # Try executing the code and capture any exception
    error = None
    try:
        exec(code_string)
    except Exception as e:
        error = e
    
    # Restore the original stdout
    sys.stdout = old_stdout
    
    # Get the captured output
    captured_output = new_stdout.getvalue()
    if isinstance(captured_output, str):
        captured_output = captured_output.strip()
    
    # Return the captured output or error
    return captured_output, error


In [ ]:
task_name = "mini_MathVista_grid"
class Args:
    def __init__(self, task_name, attacking_model, captioning_model):
        # Input
        self.poison_data_dir = f'data/poisons/{task_name}+{method}'
        self.input_file = 'questions.json'
        self.task_data_pth = f'data/{task_name}'

        # Output
        self.output_dir = f'results/{attacking_model}'
        self.output_file = f'output_{attacking_model}.json'

        # Model
        self.model = attacking_model
        self.key = ""

        # Query
        self.query_file = None
        # self.caption_file = '../data/texts/captions_bard.json'
        # self.ocr_file = '../data/texts/ocrs_easyocr.json'
        self.shot_type = 'solution'
        self.shot_num = 0
        self.use_caption = False
        self.use_ocr = False

        # Other settings
        self.rerun = False
        self.debug = False

    def __repr__(self):
        return f"Args({self.__dict__})"

args = Args(task_name, attacking_model, captioning_model)

In [ ]:

    # load data
    input_file = os.path.join(args.task_data_pth, args.input_file)
    print(f"Reading {input_file}...")
    data = read_json(input_file)
    # load or create query data
    if args.query_file:
        query_file = os.path.join(args.task_data_pth, args.query_file)
        if os.path.exists(query_file):
            print(f"Loading existing {query_file}...")
            query_data = read_json(query_file)
    else:
        print("\nCreating new query...")
        # load caption
        caption_data = {}
        if args.use_caption:
            caption_file = args.caption_file
            if os.path.exists(caption_file):
                print(f"Reading {caption_file}...")
                try:
                    caption_data = read_json(caption_file)["texts"]
                    print("Caption data loaded.")
                except:
                    print("Caption data not found!! Please Check.")                    
        # load ocr
        ocr_data = {}
        if args.use_ocr:
            ocr_file = args.ocr_file
            if os.path.exists(ocr_file):
                print(f"Reading {ocr_file}...")
                try:
                    ocr_data = read_json(ocr_file)["texts"]
                    print("OCR data loaded.")
                except:
                    print("OCR data not found!! Please Check.")
        # create query
        query_data = create_query_data(data, caption_data, ocr_data, args)

    # output file
    os.makedirs(args.output_dir, exist_ok=True)
    output_file = os.path.join(args.output_dir, args.output_file)
    
    # load results
    if os.path.exists(output_file):
        print("\nResults already exist.")
        print(f"Reading {output_file}...")
        results = read_json(output_file)
    else:
        results = {}

    # load model
    print(f"\nLoading {args.model}...")
    if args.model == 'bard':
        if args.key == '':
            print("Loading key from environment variable")
            key = os.environ['_BARD_API_KEY']
        else:
            key = args.key
        model = bard.Bard_Model(key)
    
    elif "gpt" in args.model:
        if args.key == '':
            print("Loading token from environment variable")
            key = os.getenv("OPENAI_API_KEY")
        else:
            key = args.key
        model = gpt.GPT_Model(args.model, key)
    
    elif "claude" in args.model:
        if args.key == '':
            print("Loading token from environment variable")
            key = os.environ.get("ANTHROPIC_API_KEY")
        else:
            key = args.key
        model = claude.Claude_Model(args.model, key)
    elif "internlm" in args.model:
        model = internlm.InternLM_Model()
    elif "llava_one_v" in args.model:
        model = llava_one_v.Llava_One_V()
    elif "internvl" in args.model:
        model = internvl.InternVL_Model()
            
    
    print(f"Model loaded.")
    
    # build final test pid list
    test_pids = list(data.keys())
    print("\nNumber of test problems in total:", len(test_pids))

    available_directories = [d for d in os.listdir(args.poison_data_dir) if os.path.isdir(os.path.join(args.poison_data_dir, d))]
    target_names = read_json(os.path.join(args.task_data_pth, "target/caps.json"))
    if not all([name in available_directories for name in target_names]):
        print("Not all targets have directories. Working with:", available_directories)
    target_names = [item["name"] for item in target_names["annotations"] if item["name"] in available_directories]

    skip_pids = []
    if not args.rerun:
        print("\nRemoving problems with existing valid response...")
        for i, name in enumerate(target_names):
            skip_pids.append([])
            for pid in test_pids:
                # print(f"Checking {pid}...")
                if pid in results and 'response' in results[pid]:
                    response = results[pid][name]['response']
                    if verify_response(response):
                        # print(f"Valid response found for {pid}.")
                        skip_pids[i].append(pid)
    else:
        print("\nRerun answer extraction for all problems...")

    test_pids = [[pid for pid in test_pids if pid not in target_skip_pids] for target_skip_pids in skip_pids]
    print("Number of test problems to run for each target:", {target_names[i]: len(target_pids) for i, target_pids in enumerate(test_pids)})
    # print(test_pids)

    

    # tqdm, enumerate results
    for i, target_name in enumerate(target_names):
        for _, pid in enumerate(tqdm(test_pids[i])):
            problem = data[pid]
            query = query_data[pid]
            image = problem['image']
            image_path = os.path.join(args.poison_data_dir, target_name, f'{int(image.split(".")[0])+1}.png')

            if args.debug:
                print("--------------------------------------------------------------")
            print(f"\nGenerating response for {pid}...")
            try:
                response = model.get_response(image_path, query)
                new_caption = model.get_response(image_path, "describe what is in this image")
                # print(f"Response: {response}")
                if pid not in results:
                    results[pid] = problem
                if "targets" not in results[pid]:
                    results[pid]["targets"] = {}
                if target_name not in results[pid]["targets"]:
                    results[pid]["targets"][target_name] = {}

                results[pid]["targets"][target_name]['query'] = query
                results[pid]["targets"][target_name]['model_description'] = new_caption
                if args.shot_type == 'solution':
                    results[pid]["targets"][target_name]['response'] = response
                else:
                    output, error = evaluate_code(response)
                    results[pid]["targets"][target_name]['response'] = response
                    results[pid]["targets"][target_name]['execution'] = output
                    results[pid]["targets"][target_name]['error'] = str(error)
                if args.debug:
                    print(f"\n#Query: \n{query}")
                    print(f"\n#Response: \n{response}")
            except Exception as e:
                print(e)
                print(f"Error in extracting answer for {pid}")
                results[pid][target_name]['error'] = e
        
            try:
                print(f"Saving results to {output_file}...")
                save_json(results, output_file)
                print(f"Results saved.")
            except Exception as e:
                print(e)
                print(f"Error in saving {output_file}")

Reading data/mini_MathVista_grid/questions.json...

Creating new query...

Results already exist.
Reading results/internvl/output_internvl.json...

Loading internvl...
FlashAttention2 is not installed.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model loaded.

Number of test problems in total: 100
Not all targets have directories. Working with: ['bar', 'docs', 'tbl', 'synth', 'nat', 'pie', 'sci', 'vlin', 'rdar', 'pzle', 'med', 'sctr', 'func', 'line', 'map', 'abst', 'heat']

Removing problems with existing valid response...
Number of test problems to run for each target: {'bar': 100, 'docs': 100, 'tbl': 100, 'synth': 100, 'nat': 100, 'pie': 100, 'sci': 100, 'vlin': 100, 'rdar': 100, 'pzle': 100, 'med': 100, 'sctr': 100, 'func': 100, 'line': 100, 'map': 100, 'abst': 100, 'heat': 100}


  0%|          | 0/100 [00:00<?, ?it/s]

...

Saving results to results/internvl/output_internvl.json...
Results saved.

Generating response for 31...


 32%|███▏      | 32/100 [04:19<09:45,  8.62s/it]

Saving results to results/internvl/output_internvl.json...
Results saved.

Generating response for 32...


 33%|███▎      | 33/100 [04:28<09:53,  8.86s/it]

Saving results to results/internvl/output_internvl.json...
Results saved.

Generating response for 33...


### Extract Answer

In [ ]:
import os
import re
import time
import argparse

from tqdm import tqdm
from poison_utils import *

# OpenAI
import openai
from dotenv import load_dotenv
from prompts.ext_ans import demo_prompt

In [ ]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def verify_extraction(extraction):
    extraction = extraction.strip()
    if extraction == "" or extraction == None:
        return False
    return True


def create_test_prompt(demo_prompt, query, response):
    demo_prompt = demo_prompt.strip()
    test_prompt = f"{query}\n\n{response}"
    full_prompt = f"{demo_prompt}\n\n{test_prompt}\n\nExtracted answer: "
    return full_prompt


def extract_answer(response, problem, target_name, model=None, quick_extract=False):
    question_type = problem['question_type']
    answer_type = problem['answer_type']
    choices = problem['choices']
    query = problem[target_name]['query']
    pid = problem['pid']

    if response == "":
        return ""
    
    if question_type == 'multi_choice' and response in choices:
        return response
    
    if answer_type == "integer":
        try:
            extraction = int(response)
            return str(extraction)
        except:
            pass

    if answer_type == "float":
        try:
            extraction = str(float(response))
            return extraction
        except:
            pass

    # quick extraction
    if quick_extract:
        print("Quickly extracting answer...")
        # The answer is "text". -> "text"
        try:
            result = re.search(r'The answer is "(.*)"\.', response)
            if result:
                extraction = result.group(1)
                return extraction
        except:
            pass

    # general extraction
    try:
        full_prompt = create_test_prompt(demo_prompt, query, response)
        # extraction = model.get_chat_response(full_prompt)
        extraction = get_chat_response(full_prompt, openai.api_key)
        return extraction
    except Exception as e:
        print(e)
        print(f"Error in extracting answer for {pid}")

    return ""

    

In [ ]:
args = {
    "output_dir": f"results/{model}",
    "output_file": f"output_{model}.json",
    "response_label": ,
    "llm_engine": ,
    "number": ,
    "quick_extract": ,
    "rerun": , 
    "save_every": ,
    "output_label": 
}

In [ ]:
# args
label = args.response_label
result_file = os.path.join(args.output_dir, args.output_file)

if args.output_label != '':
    output_file = result_file.replace('.json', f'_{args.output_label}.json')
else:
    output_file = result_file

# read results
print(f"Reading {result_file}...")
results = read_json(result_file)

# model = llama_3_1.LLaMA()

# full pids
full_pids = list(results.keys())
if args.number > 0:
    full_pids = full_pids[:min(args.number, len(full_pids))]
print("Number of testing problems:", len(full_pids))

# test pids
if args.rerun:
    test_pids = full_pids
else:
    test_pids = []
    for pid in full_pids:
        # print(pid)
        if 'extraction' not in results[pid] or not verify_extraction(results[pid]['extraction']):
            test_pids.append(pid)


test_num = len(test_pids)
print("Number of problems to run:", test_num)
# print(test_pids)

# tqdm, enumerate results
for i, pid in enumerate(tqdm(test_pids)):
    target_names = results[pid]["targets"].keys()
    for name in target_names:
        problem = results[pid]

        assert label in problem["targets"][name]
        response = problem["targets"][name][label]       

        
        # extraction  = extract_answer(response, problem, name, model, args.quick_extract)
        extraction  = extract_answer(response, problem, name, None, args.quick_extract)
        results[pid]["targets"][name]['extraction'] = extraction

        if i % args.save_every == 0 or i == test_num - 1:
            print(f"Saving results to {output_file}...")
            save_json(results, output_file)
            print(f"Results saved.")